In [11]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import seaborn as sns
import os
import os.path as osp
import sys
import pickle
import joblib
from collections import Counter
from itertools import product
import torch
import pdb
import random
import tables
from sklearn.linear_model import LogisticRegression, LinearRegression
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, average_precision_score,\
balanced_accuracy_score
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.calibration import CalibratedClassifierCV
import heckman
from heckman import Heckman
import wandb
from wandb.lightgbm import wandb_callback, log_summary
from dill.source import getsource
from dill import detect
import functools

### set the seeds and change to current directory + set the output directory

In [12]:
SEED=90210
np.random.seed(SEED)
os.environ['USER_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/'
os.environ['OUT_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory'

### Create function to pickle functions

In [13]:
def function_to_string(fn):
    return getsource(detect.code(fn)) 


In [14]:
processed_data_path=osp.join(os.environ.get('OUT_PATH'), 'stop_and_frisk')


In [15]:
os.environ['WANDB_SILENT']="true"


In [16]:
os.chdir('/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/')
from AnalysisFuncs import plotCorr, getPred_fromProb, getMetrics, loadModelData, getCorr, getGroundTruth, getURange
from AnalysisFuncs import getResiduals, plotDistributionProbs, plotCorr_w_Unobs
from AnalysisFuncs import saveFile, loadFile
from AnalysisFuncs import hypertune


In [17]:
train_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_X.npy')
train_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_y_T.npy')
val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/val_X.npy')
val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/val_y_T.npy')
train_cross_val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_X.npy')
train_cross_val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_y_T.npy')
test_X = loadFile(osp.join(processed_data_path,'predict_T'), '/test_X.npy')
test_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/test_y_T.npy')

train_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_X_D_given_T.npy')
train_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_y_D_given_T.npy')
val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_X_D_given_T.npy')
val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_y_D_given_T.npy')
train_cross_val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_X_D_given_T.npy')
train_cross_val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_y_D_given_T.npy')
test_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_X_D_given_T.npy')
test_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_y_D_given_T.npy')



train_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/train_y_D_and_T.npy')
val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/val_y_D_and_T.npy')
train_cross_val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), 
                                       '/train_cross_val_y_D_and_T.npy')
test_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/test_y_D_and_T.npy')


In [18]:
# load models
LR_model=LogisticRegression()
LGBM_model = LGBMClassifier()


In [9]:
#hypertune with LR model
params_dict_LR = dict(C=stats.uniform(loc=0, scale=2), penalty=['l2', 'l1'], solver=['liblinear', 'saga'])
bst_params_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X, train_cross_val_y_T, cv=5)


/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/li

cv results:{'mean_fit_time': array([625.18709998, 334.47634478, 611.60890169,  44.2617507 ,
       262.11387615, 575.58005772, 341.73333969, 248.92215061,
       179.81642618, 516.66639342]), 'std_fit_time': array([107.28480312,   4.39592883,  99.30881057,   2.64970805,
         7.59043118,  47.22427155,   0.40618852,  35.00631705,
         9.76282579,  72.19689247]), 'mean_score_time': array([0.17860103, 0.14254522, 0.09908285, 0.10485511, 0.10860624,
       0.10156932, 0.09996457, 0.09941158, 0.09510589, 0.09593096]), 'std_score_time': array([0.06716507, 0.05951776, 0.00235774, 0.00423124, 0.0069171 ,
       0.00457084, 0.00253655, 0.00075374, 0.00077765, 0.00216726]), 'param_C': masked_array(data=[1.0976270078546495, 1.2055267521432877,
                   0.8473095986778094, 0.875174422525385,
                   1.9273255210020586, 1.5834500761653292,
                   1.1360891221878646, 0.14207211639577388,
                   0.04043679488065144, 1.556313501899701],
             

In [ ]:
bst_params_D_given_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X_D_given_T,
                                    train_cross_val_y_D_given_T, cv=5)
bst_params_D_and_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X, train_cross_val_y_D_and_T, cv=5)


/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/rr568/.conda/envs/icu-benchmark/li

In [ ]:
# save the bst params dict
saveFile(osp.join(processed_data_path, 'predict_T'), bst_params_T_LR, 'bst_params_T_LR.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_given_T'), bst_params_D_given_T_LR, 'bst_params_D_given_T_LR.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_and_T'), bst_params_D_and_T_LR, 'bst_params_D_and_T_LR.pkl')


In [19]:
#hypertune with LGBM model
params_dict_LGBM = dict(max_depth=[3,4,5,6,7], 
                        feature_fraction=stats.uniform(loc=0.5, scale=0.5), 
                        bagging_fraction=stats.uniform(loc=0.7, scale=0.3),
                        min_child_samples=[10,20,30,50])
bst_params_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X, train_cross_val_y_T, cv=5)
bst_params_D_given_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X_D_given_T,
                                    train_cross_val_y_D_given_T, cv=5)
bst_params_D_and_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X, train_cross_val_y_D_and_T, cv=5)


[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Curre

[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Cur

[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Cur

In [20]:
# frozen object rv=stats.uniform(0,4)
# rv.rvs(size=10)
# array([2.08306298, 3.48690451, 3.15051837, 3.01309381, 3.98904335,
#        3.89423938, 3.04271042, 3.16679703, 0.02618609, 0.72705644]) 


In [21]:
# save the bst params dict
saveFile(osp.join(processed_data_path, 'predict_T'), bst_params_T_LGBM, 'bst_params_T_LGBM.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_given_T'), bst_params_D_given_T_LGBM, 'bst_params_D_given_T_LGBM.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_and_T'), bst_params_D_and_T_LGBM, 'bst_params_D_and_T_LGBM.pkl')